# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [108]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sqlalchemy import create_engine
nltk.download(['punkt', 'wordnet'])
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /Users/nicholas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nicholas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [109]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df =  pd.read_sql('Disaster_messages', con=engine)
# text values
X = df.message
# category values
Y = df.iloc[:, 4:]

In [110]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [111]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [112]:
from sklearn.multioutput import MultiOutputClassifier
moc = MultiOutputClassifier(RandomForestClassifier())
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', moc)
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
model = []
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [114]:
def results(y_test, y_pred):
    df_y_pred = pd.DataFrame(y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column], df_y_pred[column]))

In [115]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
y_pred = model.predict(X_test)

In [116]:
df_y_pred = pd.DataFrame(y_pred, columns = y_test.columns)

In [117]:
results(y_test, y_pred)

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.62      0.38      0.47      1530
           1       0.83      0.93      0.87      4985
           2       0.55      0.31      0.39        39

    accuracy                           0.80      6554
   macro avg       0.66      0.54      0.58      6554
weighted avg       0.78      0.80      0.78      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.88      0.98      0.93      5446
           1       0.82      0.37      0.51      1108

    accuracy                           0.88      6554
   macro avg       0.85      0.68      0.72      6554
weighted avg       0.87      0.88      0.86      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6481
           1       0.00      0.00      0.00        73

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

------------------------------------------------------

FEATURE: shops

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6534
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      1.00      6554

------------------------------------------------------

FEATURE: aid_centers

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.00      0.00      0.00        84

    accuracy                    

### 6. Improve your model
Use grid search to find better parameters. 

In [118]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x14a723320>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
            

In [128]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__

In [135]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
             'clf__estimator__min_samples_leaf': [2, 5, 10]
             }

gs = GridSearchCV(model, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [136]:
gs.fit(X_train, y_train)
y_pred = gs.predict(X_test)

In [137]:
results(y_test, y_pred)

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.86      0.01      0.02      1530
           1       0.76      1.00      0.87      4985
           2       0.00      0.00      0.00        39

    accuracy                           0.76      6554
   macro avg       0.54      0.34      0.30      6554
weighted avg       0.78      0.76      0.66      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5446
           1       1.00      0.00      0.00      1108

    accuracy                           0.83      6554
   macro avg       0.92      0.50      0.45      6554
weighted avg       0.86      0.83      0.75      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0     

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6534
           1       0.00      0.00      0.00        20

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      1.00      6554

------------------------------------------------------

FEATURE: aid_centers

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6470
           1       0.00      0.00      0.00        84

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

------------------------------------------------------

FEATURE: other_infrastructure

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6252
           1       0.00      0.00      0.00       302

    accuracy     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [139]:
# tbc

### 9. Export your model as a pickle file

In [138]:
import pickle
pickle.dump(gs, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.